In [ ]:
import cv2
import os
import numpy as np

# Create training data

In [ ]:
def create_training_data(name, label, save_folder, num_samples):
    '''
    Creates the training data set by capturing a certain number of face samples from your video camera.
    The face detection model used for this part is the Haar Cascade model.
    '''
    face_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + "haarcascade_frontalface_default.xml" )

    # Initialize and start realtime video capture
    cam = cv2.VideoCapture(0)
    cam.set(3, 640) # set video width
    cam.set(4, 480) # set video height
    
    count = 0

    while True:
        ret, img = cam.read()

        if ret == True:
            gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(gray_scale, 1.2, 4, minSize=(30, 30))

            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                count += 1

                # Save the image 
                cv2.imwrite(save_folder + "/" + label + "_" + name + "_" + str(count) + ".jpg", gray_scale[y:y+h, x:x+w])
                cv2.imshow('Create Training Data', img)
        
        k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= int(num_samples): # Take num_samples face samples and stop video
            break

    cam.release()
    cv2.destroyAllWindows()
    print("Finished collecting samples.")


In [ ]:
name = input("Please enter the name of the person being recorded: ")
label = input("Please enter a label for this person: ")
num_samples = input("Please enter the number of samples you would like to collect: ")

create_training_data(name=name, label=label, save_folder="training_data", num_samples=num_samples)

# Load data

In [ ]:
def load_data(data_folder):
    data = os.listdir(data_folder)

    names = []
    labels = []
    faces = []

    for img_name in data:
        name = img_name.split('_')[1]
        label = int(img_name.split('_')[0])

        image_path = data_folder + "/" + img_name

        img = cv2.imread(image_path)

        gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        names.append(name)
        labels.append(label)
        faces.append(cv2.resize(gray_scale, (254, 254)))

    return names, faces, labels

In [ ]:
print("Loading training data...")
names, faces_raw, labels = load_data("training_data")
names = list(dict.fromkeys(names))
faces = np.array(faces_raw) / 255
print("Data prepared")

print(f"Number of names: {len(names)}")
print(f"Number of faces: {len(faces)}")
print(f"Number of labels: {len(labels)}")

number_of_classes = len(names)

# LBPH Face Recognizer model

In [ ]:
def train_lbph_recognizer(faces, labels):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(faces, np.array(labels))

    return recognizer

# Train the face recognizer model and save the model
print("Training LBPH Face Recognizer model...")
lbph_model = train_lbph_recognizer(faces, labels)
print("Done training")

# VGG16 model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.optimizers import Adam

vgg_model = Sequential()
vgg_model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu", input_shape=(254,254,1)))
vgg_model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
vgg_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
vgg_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
vgg_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
vgg_model.add(Flatten())
vgg_model.add(Dense(units=4096,activation="relu"))
vgg_model.add(Dense(units=4096,activation="relu"))
vgg_model.add(Dense(units=number_of_classes, activation="softmax"))

vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

vgg_model.summary()

In [ ]:
from keras.utils import to_categorical

y_data = to_categorical(np.array(labels), number_of_classes)
train_data = faces.reshape(np.array(faces).shape[0], 254, 254, 1)

vgg_model.fit(train_data, y_data, epochs=15, sbatch_size=32)


# AlexNet model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout

alexnet_model = Sequential()

# Layer 1
alexnet_model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(254, 254, 1)))
alexnet_model.add(MaxPool2D((3, 3), strides=(2, 2)))

# Layer 2
alexnet_model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
alexnet_model.add(MaxPool2D((3, 3), strides=(2, 2)))

# Layer 3
alexnet_model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 4
alexnet_model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 5
alexnet_model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))

# Flatten before dense layers
alexnet_model.add(Flatten())

# Dense layers
alexnet_model.add(Dense(4096, activation='relu'))
alexnet_model.add(Dropout(0.5))
alexnet_model.add(Dense(4096, activation='relu'))
alexnet_model.add(Dropout(0.5))
alexnet_model.add(Dense(number_of_classes, activation='softmax'))

alexnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

alexnet_model.summary()

In [ ]:
from keras.utils import to_categorical

y_data = to_categorical(np.array(labels), number_of_classes)
train_data = faces.reshape(np.array(faces).shape[0], 254, 254, 1)

alexnet_model.fit(train_data, y_data, epochs=15, batch_size=32)

# Test the different models

In [ ]:
def test_model(model, model_name, names):
    face_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + "haarcascade_frontalface_default.xml" )
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Initialize and start realtime video capture
    cam = cv2.VideoCapture(0)
    cam.set(3, 640) # set video width
    cam.set(4, 480) # set video height

    while True:
        ret, img = cam.read()
        gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray_scale, scaleFactor=1.2, minNeighbors=4, minSize=(30,30))

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            face = gray_scale[y:y+h, x:x+w]
            face_resized = np.array(cv2.resize(face, (254, 254))) / 255


            if model_name == "lbph":
                id, confidence = model.predict(face_resized)

                if (confidence < 100):
                    name = names[id]
                    confidence = "  {0}%".format(round(100 - confidence))
                else:
                    name = "unknown"
                    confidence = "  {0}%".format(round(100 - confidence))
            else:
                confidence = ""
                face_resized = np.reshape(face_resized, (1, 254, 254, 1))
                id = int(np.argmax(model.predict(face_resized), axis=-1))
                name = names[id]

            cv2.putText(
                        img,
                        name,
                        (x+5, y-5),
                        font,
                        1,
                        (255, 255, 255),
                        2
                       )
            
            cv2.putText(
                        img,    
                        str(confidence), 
                        (x+5,y+h-5), 
                        font, 
                        1, 
                        (255,255,0), 
                        1
                       ) 
        
        cv2.imshow('Face Recognition', img)
        k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break

    # Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cam.release()
    cv2.destroyAllWindows()


## LBPH model

In [ ]:
test_model(model=lbph_model, model_name="lbph", names=names)

## VGG16 model

In [ ]:
test_model(model=vgg_model, model_name="vgg", names=names)

## AlexNet model

In [ ]:
test_model(model=alexnet_model, model_name="alexnet", names=names)